# 신경망 모델 구성하기
- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성된다.
- torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
- Pytorch의 모든 모듈은 nn.Module의 하위클래스이다.
- 신경망은 다른 모듈(layer)로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기
- 가능한 경우 gpu또는 mps같은 하드웨어 가속기에서 모델을 학습하려고 한다.
- `torch.cuda` 또는 `torch.backends.mps`가 사용가능한지 확인 후 사용한다.

In [10]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(device)

cuda


## 클래스 정의하기
- 신경망 모델을 `nn.Module` 의 하위클래스로 정의하고, `__init__` 에서 신경망 계층을 초기화 한다.
- `nn.Module` 을 상속받은 모든 클래스는 `forward`메소드에 입력 데이터에 대한 연산을 구현한다.

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(
        self
    ):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear( 28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        
    def forward(
        self,
        x
    ):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork` 의 인스턴스(instance)를 생성하고 이를 `device` 로 이동한 뒤, 구조(structure)를 출력한다.

In [9]:
device

{'cuda'}

In [16]:
model = NeuralNetwork().to(device = device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- 모델을 사용하기 위해 입력 데이터를 전달받는다.
- 일부 백그라운드 연산과 함께 모델의 `forward`를 실행한다.
- `model.forward()`를 직접 호출하진 말자

- 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환한다.
- 2차원 텐서의 `dim=0`은 각 분류(class)에 대한 원시(raw) 예측값 10개가,
`dim=1`에는 각 출력의 개별 값들이 해당한다.
- 원시 예측값을 `nn.softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻을 수 있다.

In [17]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"예측된 클래스 : {y_pred}")

예측된 클래스 : tensor([7], device='cuda:0')


# 모델 계층 (layer)
- FashionMNIST 모델의 계층들을 알아보자.
- 28x28 크기의 미니 배치를 가져와서 신경망을 통과할 때 어떤 일이 생기는지 알아보자.

In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten()
- `nn.Flatten()` 계층을 초기화하여 각 28x28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열로 변환한다.
- dim=0의 미니 배치 차원은 유지된다.

In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.linear()
- 선형 계층은 저장된 가중치(w)와 편항(b)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.Relu()
- 비선형 활성화(activation)은 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만든다.
- 비선형 활성화는 선형 변환 뒤에 적용되어 비선형성(nonlinearity)를 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.
- 이 모델에서는, nn.ReLU 를 선형 계층들 사이에 사용하지만, 모델을 만들 때 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [22]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2337,  0.3501, -0.3407, -0.3966,  0.0936,  0.1964, -0.2025, -0.2019,
          0.1711,  0.1221,  0.2488,  0.3113,  0.0166,  0.1497, -0.0623, -0.1446,
          0.4969, -0.4211, -0.3085, -0.2059],
        [-0.1345,  0.4148, -0.4725, -0.4682,  0.1494,  0.2642, -0.2505, -0.4098,
          0.3958, -0.3361,  0.4115,  0.4252, -0.0855,  0.6486,  0.3119, -0.0588,
          0.2099, -0.2310, -0.2038,  0.0950],
        [-0.0099,  0.4720, -0.3243, -0.2249, -0.3307,  0.2328, -0.1322, -0.0128,
          0.1529,  0.0074,  0.3565,  0.4544, -0.2712,  0.2384,  0.2233, -0.0139,
          0.0146, -0.3342, -0.2212,  0.0305]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3501, 0.0000, 0.0000, 0.0936, 0.1964, 0.0000, 0.0000, 0.1711,
         0.1221, 0.2488, 0.3113, 0.0166, 0.1497, 0.0000, 0.0000, 0.4969, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.4148, 0.0000, 0.0000, 0.1494, 0.2642, 0.0000, 0.0000, 0.3958,
         0.0000, 0.4115, 0.4252, 0.0000, 0.6486, 0.31

## nn.Sequential()
- `nn.Sequential()`은 순서를 갖는 모듈의 컨테이너이다.
- 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.
- 순차 컨테이너(sequential container)를 사용하여 아래 `seq_modules`와 같은 신경망을 빠르게 만들 수 있다.

In [23]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

## nn.Softmax()
- 신경망의 마지막 계층인 `nn.Softmax` 모듈에 전달될 ([-무한대,+무한대])범위의 원시 값(raw)인) logits를 반환한다.
- logits는 모델의 각 분류(class)에 대한 확률을 나타내도록 ([0,1])범위로 비례하게 조정(scale)된다.
- `dim` 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [25]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0732, 0.1336, 0.0810, 0.1328, 0.0900, 0.0938, 0.0938, 0.1080, 0.0938,
         0.0999],
        [0.0760, 0.1171, 0.0877, 0.1271, 0.0767, 0.0964, 0.0993, 0.1107, 0.0900,
         0.1191],
        [0.0676, 0.1396, 0.0786, 0.1284, 0.0859, 0.0936, 0.0936, 0.1027, 0.0956,
         0.1144]], grad_fn=<SoftmaxBackward0>)


## 모델 매개변수
- 신경망 내부의 많은 계층들은 매개변수화(parameterize)된다.
- 즉, 학습 중에 최적화되는 가중치(w)와 편향(b)에 연관지어진다.
- `nn.Module`을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(trace)되며, 모델의 `parameter()` 및
`named_parameter()` 메소드로 모든 매개변수에 접근할 수 있게 된다.
- 아래 예제에서 각 매개변수들을 순회(iterate)하며, 매개변수의 크기와 값을 출력해보자.

In [27]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Value: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Value: tensor([[ 0.0256,  0.0168,  0.0127,  ...,  0.0251, -0.0350,  0.0131],
        [ 0.0335, -0.0348,  0.0266,  ...,  0.0087,  0.0129,  0.0347]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Value: tensor([0.0150, 0.0003], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Value: tensor([[-0.0332, -0.0273,  0.0103,  ...,  0.0089, -0.0420, -0.0271],
        [-0.0139,  0.0418, -0.0306,  ...,  0.0086, -0.0127, -0.0305]],
       device='cuda:0', grad_fn=<SliceBackwar

## Ref
[torch.nn API](https://pytorch.org/docs/stable/nn.html)